In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import os

spark = SparkSession.builder.master("local").appName("UnionName").getOrCreate()
sc = spark.sparkContext

In [3]:
# sql adaptive query execution adaptive.coalescePartitions.enabled will makr paritions dynamic
sc.setLogLevel("Error")
spark.conf.set("spark.sql.shuffle.partitions",3)
spark.conf.get("spark.sql.shuffle.partitions")
spark.conf.set("spark.sql.adaptive.enabled","false")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled","false")
spark.conf.set("spark.sql.execution.arrow.enabled",True)
spark.conf.set("spark.sql.execution.arrow.fallback.enabled",True)

"""
Merge two data frames use, of Union and UnionByName

1.Union - merge two data frame wrt to position of column and datatype of column in data

2.Merge by position and data type using union and select

3.UnionByName Merge two data frame wrt name of column in data
  Union by name works for non matching columns(df1 - 3 cols, df2 - 4 cols) in two data frames as well. Appends null
  df1.unionByName(df2,allowMissingColumns=True).show()

"""

#WithoutInferSchema
#headerTrue will read first row and assign column names but type is String for all
#spark job created to read first column
filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"
df = spark.read.option("header",True) \
            .option("inferSchema",True) \
            .option("delimiter",",") \
            .csv(filepath + "IntPersonal_transactions.csv")

In [4]:
list_a = [["2020","Dhanush","Karnan"]]
list_b = [["Vikram","2020","Master"]]

In [8]:
df1 = spark.createDataFrame(list_a,("year","cast","movie"))
df2 = spark.createDataFrame(list_b,("cast","year","movie"))

In [9]:
df2.show()

+------+----+------+
|  cast|year| movie|
+------+----+------+
|Vikram|2020|Master|
+------+----+------+



In [12]:
#Union - merge two data frame wrt to position of column and datatype of column in data
df1.union(df2).show()

+------+-------+------+
|  year|   cast| movie|
+------+-------+------+
|  2020|Dhanush|Karnan|
|Vikram|   2020|Master|
+------+-------+------+



In [16]:
#Merge by position and data type using union
in_df2 = df2.select("year","cast","movie")
df1.union(in_df2).show()

+----+-------+------+
|year|   cast| movie|
+----+-------+------+
|2020|Dhanush|Karnan|
|2020| Vikram|Master|
+----+-------+------+



In [17]:
#UnionByName Merge two data frame wrt name of column in data
#Union by name works for non matching columns in two data frames as well. Appends null
df1.unionByName(df2,allowMissingColumns=True).show()

+----+-------+------+
|year|   cast| movie|
+----+-------+------+
|2020|Dhanush|Karnan|
|2020| Vikram|Master|
+----+-------+------+

